<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/Run_KGDTIs_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [3]:
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [4]:
pip install ampligraph

In [5]:
import ampligraph as ampligraph

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import ampligraph as ampligraph
from ampligraph.datasets import load_from_csv

In [7]:
# from ampligraph.evaluation import train_test_split_no_unseen,generate_corruptions_for_fit
# # from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.datasets import load_from_csv


In [8]:
# from ampligraph.evaluation import evaluate_performance
# As of version 1.1.1, Ampligraph removed the 'evaluate_performance' function and instead introduced the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate metrics for evaluating model performance.
# If you are using version 2.0.1, you should be able to use the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate the desired metrics. Here's an example of how you can do this
from ampligraph.evaluation import mrr_score

In [9]:
from ampligraph.evaluation import mrr_score, hits_at_n_score ,mr_score

In [10]:
# from ampligraph.evaluation.common import generate_corruptions
from ampligraph.latent_features.layers.corruption_generation import CorruptionGenerationLayerTrain

In [11]:
from ampligraph.latent_features.layers.scoring import ComplEx

In [12]:
from ampligraph.latent_features.layers.scoring import TransE

In [13]:
from ampligraph.latent_features.layers.scoring import DistMult

In [14]:
# from ampligraph.utils import save_model,restore_model
from ampligraph.utils import save_model
from ampligraph.utils import restore_model

In [15]:
from tensorflow.python.keras.models import Model

In [16]:
from tensorflow.python.keras.layers import Dense, Dropout

In [17]:
from keras.layers import LSTM, Lambda, Layer, Dropout

In [18]:
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad,Adamax

In [19]:
from tensorflow import keras

In [20]:
#load data
################################################################

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
#data example: yamanishi_08
dt_08 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt',delimiter='\t',header=None)
# the script reads a csv file using pandas' read_csv function. This function reads the file from the specified path, which in this case is
# /content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt.

dt_08.columns = ['head','relation','tail']
# the columns of the DataFrame dt_08 are set using the columns attribute. The column names are 'head', 'relation', and 'tail'.

In [22]:
#kg
# ##This code is written in Python using the pandas library.
# #The goal of this code is to load two text files,
# which contain Knowledge Graph (KG) data, and concatenate them into a single pandas DataFrame.
# The KG data in these text files consists of triples (head, relation, tail), which are essentially edges in a graph.
# The 'head' is the subject, the 'relation' is the predicate, and the 'tail' is the object.

kg1 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/kegg_kg.txt',delimiter='\t',header=None)
# The pd.read_csv() function reads the specified file and creates a DataFrame. The delimiter='\t' argument tells pandas to use tabs as separators.
# The header=None argument tells pandas that the first row of the file does not contain column names.

kg2 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/yamanishi_uniprot_kg.txt',delimiter='\t',header=None)
#This code is similar to the previous one.

kg = pd.concat([kg1,kg2])
#Concatenate the two DataFrames.
#The pd.concat() function concatenates the input DataFrames into a single DataFrame.

kg.index = range(len(kg))
#Reset the index of the concatenated DataFrame.
#The index attribute of a DataFrame represents the index of the rows.
#This line of code resets the index of the concatenated DataFrame so that it starts from 0 and increments by 1.

kg.columns = ['head','relation','tail']
#Set the column names of the concatenated DataFrame.
#This line of code assigns new column names to the concatenated DataFrame.


#The resulting kg DataFrame contains the combined KG data from both text files.
# The DataFrame has three columns: 'head', 'relation', and 'tail'. The rows represent the triples (head, relation, tail) in the KG.

In [23]:
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
pip install networkx matplotlib


In [25]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


In [26]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


# اولین دو خط، دو شیء LabelEncoder را ایجاد می‌کند.
# این اشیاء برای تبدیل متغیرهای دسته‌ای به یک فرمت عددی که برای الگوریتم‌های یادگیری ماشین قابل فهم باشد، استفاده می‌شوند.
# تابع LabelEncoder() دو بار فراخوانی می‌شود تا دو شیء head_le و tail_le ایجاد شوند.
head_le = LabelEncoder()
tail_le = LabelEncoder()
relation_le=LabelEncoder()
# متد fit() بر روی هر دو شیء فراخوانی می‌شود. این متد پارامترهای لازم برای انجام رمزگذاری را محاسبه می‌کند.
head_le.fit(dt_08['head'].values)
tail_le.fit(dt_08['tail'].values)
relation_le.fit(dt_08['relation'].values)
# MinMaxScaler از ماژول preprocessing کتابخانه sklearn وارد می‌شود. این برای مقیاس‌بندی داده‌ها استفاده می‌شود.
mms = MinMaxScaler(feature_range=(0, 1))


In [27]:
dt_08['head'] = head_le.transform(dt_08['head'].values)
dt_08['tail'] = tail_le.transform(dt_08['tail'].values)
dt_08['relation']=relation_le.transform(dt_08['relation'].values)
print("نتیجه‌ی Label Encoding:")
print(dt_08)

نتیجه‌ی Label Encoding:
      head  relation  tail
0        0         0     0
1      181         0     0
2       11         0     1
3       60         0     1
4        5         0     3
...    ...       ...   ...
5122    55         0   943
5123    79         0   943
5124   335         0   943
5125   210         0   986
5126    63         0   987

[5127 rows x 3 columns]


In [28]:
dt_08 = mms.fit_transform(dt_08)
print("\nنتیجه‌ی مقیاس‌بندی با MinMaxScaler:")
print(dt_08)


نتیجه‌ی مقیاس‌بندی با MinMaxScaler:
[[0.         0.         0.        ]
 [0.22911392 0.         0.        ]
 [0.01392405 0.         0.00101215]
 ...
 [0.42405063 0.         0.95445344]
 [0.26582278 0.         0.99797571]
 [0.07974684 0.         0.99898785]]


In [29]:
#فراخوانی شناسه داروها (Drug IDs):
fp_id = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/791drug_struc.csv')['drug_id']

# فراخوانی شناسه پروتئین‌ها و توالی‌های آنها:
df_proseq = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/989proseq.csv')
df_proseq.columns = ['pro_id','pro_ids','seq']

# استخراج شناسه پروتئین‌ها:
pro_id = df_proseq['pro_id']

#فراخوانی ویژگی‌های داروها:
drug_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/morganfp.txt',delimiter=',')

#فراخوانی ویژگی‌های پروتئین‌ها:
pro_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/pro_ctd.txt',delimiter=',')

#مقیاس‌بندی ویژگی‌های پروتئین‌ها:
pro_feats_scaled = mms.fit_transform(pro_feats)

# کاهش ابعاد ویژگی‌های پروتئین با استفاده از PCA:
pro_feats_scaled2 = PCA(n_components=100).fit_transform(pro_feats_scaled)

#دوباره مقیاس‌بندی ویژگی‌های کاهش‌یافته:
pro_feats_scaled3 = mms.fit_transform(pro_feats_scaled2)

# ترکیب شناسه‌های داروها با ویژگی‌های آنها:
fp_df = pd.concat([fp_id,pd.DataFrame(drug_feats)],axis=1)

#ترکیب شناسه‌های پروتئین‌ها با ویژگی‌های آنها:
prodes_df = pd.concat([pro_id,pd.DataFrame(pro_feats_scaled3)],axis=1)


In [30]:
data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'


def load_data(i):
    # Read the train_fold csv file. The label is included.
    train = pd.read_csv(data_path+'/train_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Select only the positive examples (label == 1) from the train set.
    train_pos = train[train['label']==1]

    # Read the test_fold csv file. The label is included.
    test = pd.read_csv(data_path+'/test_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Merge the positive train examples and the knowledge graph into a single dataframe.
    data = pd.concat([train_pos,kg])[['head','relation','tail']]


    # Return the train, train_pos, test, and data dataframes.
    return train,train_pos,test,data



In [31]:
def roc_auc(y,pred):

  # این خط تابع roc_curve را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
 #تابع منحنی_roc منحنی ROC را برای مسئله طبقه بندی باینری داده شده، نرخ مثبت کاذب (FPR)، نرخ مثبت واقعی (TPR) و آستانه ها را محاسبه می کند.
    fpr, tpr, thresholds = metrics.roc_curve(y, pred)

    #roc_auc = metrics.auc(fpr, tpr): این خط تابع auc را از ماژول متریک با پارامترهای fpr و tpr فراخوانی می کند.
 # تابع auc مساحت زیر منحنی ROC را محاسبه می‌کند که امتیاز AUC-ROC است.
    roc_auc = metrics.auc(fpr, tpr)

    #return roc_auc: این خط امتیاز AUC-ROC محاسبه شده را برمی گرداند.
    return roc_auc


In [32]:
from sklearn import metrics

def aupr(y, pred):
    # این خط تابع precision_recall_curve را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع منحنی Precision-Recall را برای مسئله طبقه بندی باینری محاسبه می کند و دقت، بازیابی و آستانه ها را برمی گرداند.
    precision, recall, thresholds = metrics.precision_recall_curve(y, pred)

    # این خط تابع auc را از ماژول متریک با پارامترهای recall و precision فراخوانی می کند.
    # تابع auc مساحت زیر منحنی Precision-Recall را محاسبه می کند که امتیاز AUPR است.
    aupr_value = metrics.auc(recall, precision)

    # این خط امتیاز AUPR محاسبه شده را برمی گرداند.
    return aupr_value

In [33]:
from sklearn import metrics

def f1_score_custom(y, pred):
    # این خط تابع f1_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع F1 Score را برای مسئله طبقه بندی باینری محاسبه می کند.
    f1 = metrics.f1_score(y, pred)

    # این خط امتیاز F1 محاسبه شده را برمی گرداند.
    return f1

In [34]:
from sklearn import metrics

def precision_custom(y, pred):
    # این خط تابع precision_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع دقت را برای مسئله طبقه بندی باینری محاسبه می کند.
    precision = metrics.precision_score(y, pred)

    # این خط امتیاز دقت محاسبه شده را برمی گرداند.
    return precision

In [35]:
from sklearn import metrics

def accuracy_custom(y, pred):
    # این خط تابع accuracy_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع دقت را برای مسئله طبقه بندی باینری محاسبه می کند.
    accuracy = metrics.accuracy_score(y, pred)

    # این خط امتیاز دقت محاسبه شده را برمی گرداند.
    return accuracy

In [36]:
from sklearn import metrics

def mcc_custom(y, pred):
    # این خط تابع matthews_corrcoef را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع MCC را برای مسئله طبقه بندی باینری محاسبه می کند.
    mcc = metrics.matthews_corrcoef(y, pred)

    # این خط امتیاز MCC محاسبه شده را برمی گرداند.
    return mcc

In [37]:
from sklearn import metrics

def sensitivity_custom(y, pred):
    # این خط تابع recall_score را از ماژول متریک (بخشی از کتابخانه Scikit-Learn) با پارامترهای y و pred فراخوانی می کند.
    # این تابع حساسیت (Sensitivity) را برای مسئله طبقه بندی باینری محاسبه می کند.
    sensitivity = metrics.recall_score(y, pred)

    # این خط امتیاز حساسیت محاسبه شده را برمی گرداند.
    return sensitivity


In [38]:
from sklearn import metrics

def specificity_custom(y, pred):
    # این خط ماتریس درهم‌ریختگی (Confusion Matrix) را با استفاده از y و pred محاسبه می‌کند.
    cm = metrics.confusion_matrix(y, pred)

    # این خط عناصر ماتریس درهم‌ریختگی را به ترتیب برای TN، FP، FN و TP استخراج می‌کند.
    tn, fp, fn, tp = cm.ravel()

    # این خط ویژگی (Specificity) را با استفاده از TN و FP محاسبه می‌کند.
    specificity = tn / (tn + fp)

    # این خط امتیاز ویژگی محاسبه شده را برمی گرداند.
    return specificity


In [39]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel

In [40]:

###درج_مقیاس_گرفتن_مقیاس_شده که یک مدل تعبیه گراف دانش از پیش آموزش دیده و سه مجموعه سه تایی را به عنوان پارامترهای ورودی می گیرد.
# جاسازی های مقیاس شده را برای موضوعات و اشیاء سه گانه آموزشی و آزمایشی خروجی می دهد.
#به طور خلاصه، این تابع ابتدا تعبیه‌های موضوع و مفعول سه‌گانه‌ها را به دست می‌آورد.
#سپس، این تعبیه‌ها را به هم متصل می‌کند و تکنیک‌های عادی‌سازی و کاهش ابعاد (MinMaxScaler و PCA) را اعمال می‌کند.
#برای به دست آوردن جاسازی های مقیاس شده که می توانند برای کارهای مختلف پایین دست استفاده شوند.

# تابع را تعریف می کند و پارامترهای ورودی را مشخص می کند.
def get_scaled_embeddings(model,train_triples,test_triples,get_scaled,n_components):

    # برای هر سه‌گانه، تابع موجودیت‌های موضوع (سر) را استخراج می‌کند و جاسازی‌های آن‌ها را از مدل از پیش آموزش‌دیده دریافت می‌کند. این کار را برای سه گانه آموزشی و آزمایشی انجام می دهد.
 # [train_sub_embeddings,test_sub_embeddings] = [model.get_embeddings(x['head'].values, embedding_type='entity') برای x در [train_triples,test_triples]]
    [train_sub_embeddings,test_sub_embeddings]=[model.get_embeddings(x['head'].values, embedding_type='e')  for x in [train_triples,test_triples]]

    #به طور مشابه، موجودیت های شی (دم) را استخراج می کند و تعبیه های آنها را از مدل از پیش آموزش دیده می گیرد. باز هم این کار را برای سه گانه آموزشی و آزمایشی انجام می دهد.
    [train_obj_embeddings,test_obj_embeddings] = [model.get_embeddings(x['tail'].values, embedding_type='e') for x in [train_triples,test_triples]]

    # تابع جاسازی های موضوع و شی را برای هر سه در مجموعه آموزشی به هم متصل می کند.
    train_feats = np.concatenate([train_sub_embeddings,train_obj_embeddings],axis=1)

    #این تابع همچنین جاسازی های موضوع و شی را برای هر سه در مجموعه آزمایشی به هم متصل می کند.
    test_feats = np.concatenate([test_sub_embeddings,test_obj_embeddings],axis=1)

    #این تابع MinMaxScaler (mms) را به ویژگی‌های آموزشی پیوسته اعمال می‌کند تا مجموعه‌ای از ویژگی‌های متراکم نرمال شده را به دست آورد.
    train_dense_features = mms.fit_transform(train_feats)

    #این تابع همچنین MinMaxScaler را برای ویژگی‌های آزمایشی الحاقی اعمال می‌کند تا مجموعه‌ای از ویژگی‌های متراکم نرمال شده را به دست آورد.
    test_dense_features = mms.transform(test_feats)

    #اگر پارامتر get_scaled True باشد، تابع اقدام به اعمال تجزیه و تحلیل مؤلفه اصلی (PCA) به ویژگی‌های متراکم نرمال می‌کند.
    if get_scaled:

        # تابع یک نمونه PCA با تعداد مشخص شده مولفه (n_components) ایجاد می کند.
        pca = PCA(n_components=n_components)

        #عملکرد PCA را برای ویژگی های متراکم تمرین نرمال اعمال می کند.
        scaled_train_dense_features = pca.fit_transform(train_dense_features)

        #این عملکرد همچنین PCA را برای ویژگی‌های متراکم تست نرمال اعمال می‌کند.
        scaled_pca_test_dense_features = pca.transform(test_dense_features)

    #اگر پارامتر get_scaled False باشد، تابع از مرحله PCA می گذرد و مستقیماً ویژگی های متراکم نرمال شده را به متغیرهای خروجی اختصاص می دهد.
    else:

        #ویژگی های متراکم تمرین نرمال شده را به متغیر خروجی اختصاص می دهد.
        scaled_train_dense_features = train_dense_features

        #ویژگی های متراکم تست نرمال شده را به متغیر خروجی اختصاص می دهد.
        scaled_pca_test_dense_features = test_dense_features

    #جاسازی های مقیاس شده را برای موضوعات و موضوعات سه گانه آموزش و تست برمی گرداند.
    return scaled_train_dense_features,scaled_pca_test_dense_features




In [41]:

#به طور خلاصه، تابع get_features یک داده فریم داده، دو فریم داده fp_df و prodes_df و یک متغیر بولی use_pro می گیرد.
#ویژگی‌های دارویی را از fp_df و ویژگی‌های پروتئینی را از prodes_df با استفاده از ستون‌های سر و دم داده‌ها استخراج می‌کند.
#سپس، ویژگی ها را به صورت افقی بر اساس ارزش use_pro به هم متصل می کند. ماتریس ویژگی حاصل به عنوان خروجی تابع برگردانده می شود.
#این خط تابعی به نام get_features را تعریف می کند. به چهار پارامتر داده، fp_df، prodes_df و use_pro نیاز دارد.
def get_features(data,fp_df,prodes_df,use_pro):

    #این خط یک اتصال سمت چپ داده ها و fp_df را در ستون 'head' داده ها و ستون 'drug_id' در fp_df انجام می دهد.
 #پیوستن سمت چپ به این دلیل انجام می‌شود که می‌خواهیم همه رکوردها را از جدول سمت چپ (یعنی داده‌ها) و رکوردهای مطابقت‌شده را از جدول سمت راست (یعنی fp_df) نگه داریم.
 #سپس، کد 1025 ستون (از ستون 5 تا 1029) از دیتافریم حاصل را انتخاب می کند و با استفاده از ویژگی values ​​آن را به یک آرایه numpy تبدیل می کند.
 #نتیجه در متغیر_ویژگی های دارو ذخیره می شود.
    drug_features = pd.merge(data,fp_df,how='left',left_on='head',right_on='drug_id').iloc[:,4:1029].values

   #این خط عملیات مشابه قبلی را انجام می دهد، اما این بار داده ها را به هم می پیوندد و prodes_df را در ستون 'tail' داده و ستون 'pro_id' prodes_df را می پیوندد.
 #دوباره، کد 101 ستون (از ستون 5 تا 105) از دیتافریم حاصل را انتخاب می کند و با استفاده از ویژگی values ​​آن را به یک آرایه numpy تبدیل می کند.
 #نتیجه در متغیر pro_features ذخیره می شود.
    pro_features = pd.merge(data,prodes_df,how='left',left_on='tail',right_on='pro_id').iloc[:,4:105].values

   #این خط مقدار متغیر use_pro را بررسی می کند. اگر True باشد، با استفاده از تابع np.concatenate، drug_features و pro_features را به صورت افقی به هم متصل می کند.
 #اگر use_pro False باشد، مستقیماً ویژگی‌های دارو را به ویژگی متغیر اختصاص می‌دهد.
    if use_pro:
        feature = np.concatenate([drug_features,pro_features],axis=1)
    else:
        feature = drug_features

    #این خط ماتریس ویژگی نهایی را برمی گرداند.
    return feature


ازینجا به بعد رو متفاوت از دفعه قبلی نوشتم

In [42]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def get_cnn_input(re_train_all, re_test_all, train_feats, test_feats, train_des, test_des, embedding_dim, pca_components):
    # 1. ترکیب ویژگی‌ها و توضیحات:
    # `train_feats` و `train_des` برای مجموعه آموزش و `test_feats` و `test_des` برای مجموعه تست
    # در یک ماتریس واحد ترکیب می‌شوند.
    train_all_feats = np.concatenate([train_feats, train_des], axis=1)
    test_all_feats = np.concatenate([test_feats, test_des], axis=1)

    # 2. استانداردسازی ویژگی‌ها:
    # داده‌های ترکیب‌شده با استفاده از `MinMaxScaler` به بازه 0 تا 1 مقیاس‌بندی می‌شوند.
    mms = MinMaxScaler()
    train_all_feats_scaled = mms.fit_transform(train_all_feats)
    test_all_feats_scaled = mms.transform(test_all_feats)

    # 3. تبدیل به شکل مناسب برای CNN:
    # در اینجا داده‌ها با استفاده از `reshape` به یک آرایه سه‌بعدی تبدیل می‌شوند.
    # شکل جدید آرایه‌ها به صورت (تعداد نمونه‌ها، تعداد ویژگی‌ها، 1) خواهد بود.
    # این تغییر به دلیل سازگاری با لایه‌های `Conv1D` است که به یک ورودی سه‌بعدی نیاز دارند.
    train_all_feats_scaled = train_all_feats_scaled.reshape(-1, train_all_feats_scaled.shape[1], 1)
    test_all_feats_scaled = test_all_feats_scaled.reshape(-1, test_all_feats_scaled.shape[1], 1)

    # 4. ایجاد ورودی‌های مدل:
    # ورودی‌های مدل به شکل دیکشنری برای آموزش و تست ساخته می‌شوند.
    # `head` و `tail` با استفاده از `LabelEncoder` به اعداد تبدیل می‌شوند.
    train_model_input = {
        'head': head_le.transform(re_train_all['head'].values),
        'tail': tail_le.transform(re_train_all['tail'].values),
        'feats': train_all_feats_scaled
    }

    test_model_input = {
        'head': head_le.transform(re_test_all['head'].values),
        'tail': tail_le.transform(re_test_all['tail'].values),
        'feats': test_all_feats_scaled
    }

    # 5. بازگرداندن خروجی:
    # در نهایت، ورودی‌های آماده شده برای مدل و ستون‌های ویژگی‌ها بازگردانده می‌شوند.
    return train_model_input, test_model_input


In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_cnn_model(input_shape):
    model = Sequential()

    # اولین لایه کانولوشن یک‌بعدی
    # این لایه یک کانولوشن یک‌بعدی با 128 فیلتر و اندازه کرنل 4 را اعمال می‌کند.
    # 'relu' به عنوان تابع فعال‌سازی استفاده می‌شود تا به شبکه اجازه دهد که بتواند روابط غیرخطی را یاد بگیرد.
    # input_shape شکل ورودی را تعیین می‌کند که باید با داده‌های شما تطابق داشته باشد.
    model.add(Conv1D(128, kernel_size=4, strides=1, activation='relu', input_shape=input_shape))

    # اولین لایه pooling یک‌بعدی
    # این لایه اندازه ویژگی‌های تولید شده توسط لایه کانولوشن را با استفاده از pool_size=2 نصف می‌کند.
    model.add(MaxPooling1D(pool_size=2))

    # دومین لایه کانولوشن یک‌بعدی
    # این لایه مشابه اولین لایه کانولوشن است، اما بر روی خروجی لایه pooling اعمال می‌شود.
    model.add(Conv1D(128, kernel_size=4, strides=1, activation='relu'))

    # دومین لایه pooling یک‌بعدی
    model.add(MaxPooling1D(pool_size=2))

    # مسطح کردن خروجی
    # این لایه داده‌ها را از شکل چندبعدی به یک بردار یک‌بعدی تبدیل می‌کند
    # تا بتوانند به عنوان ورودی به لایه‌های کاملاً متصل وارد شوند.
    model.add(Flatten())

    # اولین لایه کاملاً متصل
    # این لایه به شبکه اجازه می‌دهد که یادگیری بیشتری از ویژگی‌ها داشته باشد.
    model.add(Dense(128, activation='relu'))

    # لایه Dropout
    # این لایه برای جلوگیری از بیش‌برازش (overfitting) استفاده می‌شود
    # و به طور تصادفی 20٪ از نورون‌ها را در طول هر مرحله آموزش غیرفعال می‌کند.
    model.add(Dropout(0.2))

    # لایه خروجی
    # این لایه با یک نورون و تابع فعال‌سازی 'sigmoid' خروجی را در بازه [0, 1] تولید می‌کند.
    # این خروجی برای مسائل دودویی (binary classification) مناسب است.
    model.add(Dense(1, activation='sigmoid'))

    # کامپایل کردن مدل
    # اینجا مدل با استفاده از Adam بهینه‌سازی می‌شود و از 'binary_crossentropy' به عنوان تابع هزینه استفاده می‌شود.
    # دقت (accuracy) به عنوان معیار ارزیابی مدل در طول آموزش در نظر گرفته شده است.
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [44]:
def evaluate_metrics(y_true, y_pred):
    # تبدیل پیش‌بینی‌ها به مقادیر باینری
    y_pred_binary = (y_pred > 0.5).astype(int)

    # محاسبه متریک‌ها
    cm = confusion_matrix(y_true, y_pred_binary)
    tn, fp, fn, tp = cm.ravel()

    sen = tp / (tp + fn)  # Sensitivity (Recall)
    spe = tn / (tn + fp)  # Specificity
    acc = (tp + tn) / (tp + tn + fp + fn)  # Accuracy
    precision = precision_score(y_true, y_pred_binary)
    f1 = f1_score(y_true, y_pred_binary)
    auroc = roc_auc_score(y_true, y_pred)
    aupr = average_precision_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred_binary)

    return sen, spe, acc, precision, f1, auroc, aupr, mcc



In [45]:
def train_cnn_and_evaluate(X_train, y_train, X_test, y_test, batch_size=64, epochs=2):
    # ایجاد مدل CNN
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])  # (Height, Width, Channels)
    model = create_cnn_model(input_shape)

    # آموزش مدل
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

    # پیش‌بینی بر روی داده‌های تست
    y_pred = model.predict(X_test).flatten()

    # ارزیابی مدل
    sen, spe, acc, precision, f1, auroc, aupr, mcc = evaluate_metrics(y_test, y_pred)

    # چاپ نتایج
    print(f"Sen: {sen:.4f}")
    print(f"Spe: {spe:.4f}")
    print(f"ACC: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUROC: {auroc:.4f}")
    print(f"AUPR: {aupr:.4f}")
    print(f"MCC: {mcc:.4f}")

    return model,sen, spe, acc, precision, f1, auroc, aupr, mcc,y_pred[:,0]

# استفاده از تابع train_cnn_and_evaluate برای آموزش و ارزیابی مدل
# X_train, y_train, X_test, y_test باید به درستی تعریف شوند و به تابع ارسال شوند.
# مدل را آموزش داده و ارزیابی کنید:
# model = train_cnn_and_evaluate(X_train, y_train, X_test, y_test)

In [46]:
pip install pykeen

In [47]:
import torch
from pykeen.models import DistMult
from pykeen.triples import TriplesFactory
from pykeen.training import LCWATrainingLoop
from pykeen.losses import MarginRankingLoss
from pykeen.regularizers import LpRegularizer
from pykeen.optimizers import Adam
from pykeen.stoppers import EarlyStopper
from pykeen.evaluation import RankBasedEvaluator

In [48]:
from pykeen.models import DistMult
from pykeen.triples import TriplesFactory
from pykeen.training import LCWATrainingLoop
from pykeen.optimizers import Adam

def knowledge_graph(data):
    # Define the model parameters
    k = 400  # embedding dimension
    epochs = 3  # number of training epochs
    batch_size = 10000  # batch size
    learning_rate = 1e-3
    margin = 0.5

    # Create the TriplesFactory
    triples_factory = TriplesFactory.from_labeled_triples(data.values)

    # Create the DistMult model
    model = DistMult(
        triples_factory=triples_factory,
        embedding_dim=k,
        random_seed=42,
    )

    # Use Adam optimizer
    optimizer = Adam(params=model.parameters(), lr=learning_rate)

    # Use Pairwise Hinge Loss
    loss = MarginRankingLoss(margin=margin)

    training_loop = LCWATrainingLoop(
        model=model,
        optimizer=optimizer,
        triples_factory=triples_factory,  # Pass the triples_factory here
    )


    # Train the model
    training_loop.train(
        triples_factory=triples_factory,
        num_epochs=epochs,
        batch_size=batch_size,
        use_tqdm_batch=False,
    )

    return model


In [49]:
def train(i,test_num_neg,train_num_neg,embedding_dim,n_components,use_pro,patience):

    #This function loads the train and test data for the current fold.
    train,train_pos,test,data = load_data(i)
    model=knowledge_graph(data)
    columns = ['head','relation','tail']
    re_train_all = train[columns]
    re_test_all = test[columns]
    train_label = train['label']

    #This function generates the embeddings for the train and test data.
    train_dense_features,test_dense_features = get_scaled_embeddings(model,re_train_all,re_test_all,False,n_components)







    #This function generates the additional features (chemical features, proline features, etc.) for the train and test data.
    train_des = get_features(re_train_all,fp_df,prodes_df,use_pro)
    test_des = get_features(re_test_all,fp_df,prodes_df,use_pro)



    #This function generates the input for the NFM model, which includes the combined features from the embeddings and additional features.
    feature_columns,train_model_input,test_model_input = get_cnn_input(re_train_all,re_test_all,
                                                                    train_dense_features,test_dense_features,
                                                                    train_des,test_des,
                                                                    embedding_dim,n_components)

    train_label = np.array(train_label)
    test_label = np.array(test_label)

    model,sen, spe, acc, precision, f1, auroc, aupr, mcc,pred_y= train_cnn_and_evaluate(train_model_input, train_label, test_model_input, test_label, batch_size=64, epochs=300)

    print("I am meymoon")
    return  model,sen, spe, acc, precision, f1, auroc, aupr, mcc,re_train_all,train_label,re_test_all,test_label,pred_y

In [ ]:
Sen = []
Spe = []
ACC = []
Precision = []
F1=[]
AUROC=[]
AUPR=[]
MCC=[]
for i in range(10):

  # print(i) prints the current iteration of the for loop.
    print(i)

    #train() is a function that runs the experiment for a given fold (i), given number of splits (10), given number of recommendations per user (10),
    #given number of features (50), given number of embeddings (200), given whether to use attention or not (True), and given the length of the session sequence (10).
    # It returns several metrics and arrays of user and item embeddings.
    model,sen, spe, acc, precision, f1, auroc, aupr, mcc,re_train_all,train_label,re_test_all,test_label,pred_y = train(i,10,10,50,200,True,10)  #stores the return values of the train() function into variables.
    #assign the ground truth labels (train_label and test_label) to the respective users in the train and test sets.
    re_train_all['label'] = train_label
    re_test_all['label'] = test_label

    #re_test_all['pred'] = pred_y stores the predicted ratings (pred_y) for each user in the test set.
    re_test_all['pred'] = pred_y

    #ROC.append(roc), PR.append(pr), ROC_s.append(roc_s), PR_s.append(pr_s) append the ROC, PR, ROC_s, and PR_s values to their respective lists for each fold.
    Sen.append(sen)
    Spe.append(spe)
    ACC.append(acc)
    Precision.append(precision)
    F1.append(f1)
    AUROC.append(auroc)
    AUPR.append(aupr)
    MCC.append(mcc)

#creates an empty pandas DataFrame.
stable_metrics = pd.DataFrame()

# store the ROC, PR, ROC_s, and PR_s values in the respective columns of the DataFrame.
stable_metrics['sen'] = Sen
stable_metrics['spe'] = Spe
stable_metrics['acc'] = ACC
stable_metrics['precision'] =Precision
stable_metrics['f1'] = F1
stable_metrics['auroc'] = AUROC
stable_metrics['aupr'] = AUPR
stable_metrics['mcc'] = MCC
#prints the summary statistics of the metrics.
stable_metrics.describe()


0
